In [1]:
import requests
import zipfile

url = 'https://opendart.fss.or.kr/api/fnlttSinglAcntAll.xml'
apiKey = '916939029f1c9724ef2868f26c9e81faf88abae0'

params = {
    'crtfc_key': apiKey,
    'corp_code': '00126380',
    'bsns_year': '2021',
    'reprt_code':'11011',
    'fs_div':'OFS'
}

response = requests.get(url, params=params)

with open('./test.xml', 'wb') as fp:
    fp.write(response.content)

In [3]:
from bs4 import BeautifulSoup

r = open('test.xml', 'r', encoding='UTF-8')
soup = BeautifulSoup(r, 'html.parser')

In [4]:
import pandas as pd
# 개발가이드 > 공시정보 > 고유번호 참고
raw = pd.read_xml('./test.xml')[['sj_nm','account_nm', 'thstrm_amount', 'frmtrm_amount', 'bfefrmtrm_amount']].iloc[2:]
is_BS = raw['sj_nm'] == '재무상태표'
is_CIS = raw['sj_nm'] == '손익계산서'
is_TCIS = raw['sj_nm'] == '포괄손익계산서'
is_CF = raw['sj_nm'] == '현금흐름표'
is_SCE = raw['sj_nm'] == '자본변동표'

raw[is_TCIS]

,sj_nm,account_nm,thstrm_amount,frmtrm_amount,bfefrmtrm_amount
61,포괄손익계산서,당기순이익(손실),3.097095e+13,1.561502e+13,1.535332e+13
62,포괄손익계산서,기타포괄손익,-6.132250e+11,-5.492990e+11,-8.519580e+11
63,포괄손익계산서,후속적으로 당기손익으로 재분류되지 않는 포괄손익,-6.132250e+11,-5.492990e+11,-8.519580e+11
64,포괄손익계산서,기타포괄손익-공정가치금융자산평가손익,-9.991600e+10,9.325100e+10,7.319900e+10
65,포괄손익계산서,순확정급여부채(자산) 재측정요소,-5.133090e+11,-6.425500e+11,-9.251570e+11
66,포괄손익계산서,후속적으로 당기손익으로 재분류되는 포괄손익,0.000000e+00,0.000000e+00,0.000000e+00
67,포괄손익계산서,총포괄손익,3.035773e+13,1.506572e+13,1.450136e+13


In [35]:
#공시번호 크롤링 corpCode

"""url = 'https://opendart.fss.or.kr/api/corpCode.xml?crtfc_key='
apiKey = '916939029f1c9724ef2868f26c9e81faf88abae0'

from io import BytesIO
from zipfile import ZipFile
import urllib.request
import xml.etree.ElementTree
import pandas
import csv

params = {
    'crtfc_key': apiKey,
}


with urllib.request.urlopen(url+apiKey) as zipresponse:
    with ZipFile(BytesIO(zipresponse.read())) as zipfile:
        zipfile.extractall('corpCode')"""

In [52]:
# 주식 코드를 api 코드로 변환

a = pd.read_xml('./corpCode/CORPCODE.xml')

stockCode = '005930'
is_target = a['stock_code'] == float('005930')

a[is_target]['corp_code']

79323    126380
Name: corp_code, dtype: int64

In [50]:
url = 'https://opendart.fss.or.kr/api/document.xml?crtfc_key='
apiKey = '916939029f1c9724ef2868f26c9e81faf88abae0'
rcept_no = '&rcept_no=20220308000798'

from io import BytesIO
from zipfile import ZipFile
import urllib.request
import xml.etree.ElementTree
import pandas
import csv

params = {
    'crtfc_key': apiKey,
}


with urllib.request.urlopen(url+apiKey+rcept_no) as zipresponse:
    with ZipFile(BytesIO(zipresponse.read())) as zipfile:
        zipfile.extractall('documents')

In [37]:
url = 'https://opendart.fss.or.kr/api/document.xml'

import requests
from io import BytesIO
import zipfile
import xmltodict

res = requests.get(url, params={'crtfc_key': apiKey, 'rcept_no':'20220308000798'})
data_xml = zipfile.ZipFile(BytesIO(res.content))

In [30]:
with open('./test.zip', 'wb') as fp:
    fp.write(res.content)

In [68]:
cnt = data_xml.read(data_xml.namelist()[1])

In [ ]:
#pd.read_xml('./documents/20220308000798_00761.xml', encoding='EUC-KR')

In [57]:
import xml.etree.ElementTree as et

xtree = et.parse('./documents/20220308000798.xml')
xroot = xtree.getroot()

ParseError: not well-formed (invalid token): line 6, column 29 (<string>)

In [56]:
with open('./documents/20220308000798_00761.xml', 'r') as f:
    for line in f:
        line = line.rstrip('&cr;')

In [70]:
from bs4 import BeautifulSoup

soup = BeautifulSoup(cnt, 'html.parser')

c:\Users\lunar\anaconda3\envs\pyexcel\lib\site-packages\bs4\builder\__init__.py:545: XMLParsedAsHTMLWarning: It looks like you're parsing an XML document using an HTML parser. If this really is an HTML document (maybe it's XHTML?), you can ignore or filter this warning. If it's XML, you should know that using an XML parser will be more reliable. To parse this document as XML, make sure you have the lxml package installed, and pass the keyword argument `features="xml"` into the BeautifulSoup constructor.
  warnings.warn(


In [80]:
table = soup.find_all('table')
tbody = soup.find_all('tbody')

c:\Users\lunar\anaconda3\envs\pyexcel\lib\site-packages\pandas\core\internals\construction.py:568: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  values = np.array([convert(v) for v in values])


,0
0,"[\n, [\n, [사업연도], \n, [2021년 01월 01일], \n, [ 부..."
1,"[\n, [\n, [금융위원회], \n, [], \n], \n, [\n, [한국거래..."
2,"[\n, [\n, [제출대상법인 유형 :], \n, [주권상장법인], \n], \n..."
3,"[\n, [\n, [], \n, [], \n], \n, [\n, [회 사 ..."
4,"[\n, [\n, [ 【 대표이사 등의 확인 】], \n, [], \n, [\n,..."
...,...
1048,"[\n, [\n, [(기준일 : ], \n, [2021년 12월 31일], \n, ..."
1049,"[\n, [\n, [삼성전기㈜ ], \n, [상장], \n, [1977.01.01]..."
1050,"[\n, [\n, [\n, <span>※ 별도재무제표 기준입니다. &amp;cr※ ..."
1051,"[\n, [\n, [\n, <a refno=""II. 사업의 내용|ADX-1|NEF-..."


In [81]:
tbody

[<tbody>
 <tr acopy="N" adelete="N">
 <td align="RIGHT" aupdatecont="N" class="NORMAL" height="60" rowspan="2" valign="MIDDLE" width="218">사업연도</td>
 <tu align="CENTER" aunit="PERIODFROM" aunitvalue="20210101" class="NORMAL" height="30" valign="MIDDLE" width="162">2021년 01월 01일</tu>
 <td aupdatecont="N" class="NORMAL" height="30" valign="MIDDLE" width="220"> 부터</td>
 </tr>
 <tr acopy="N" adelete="N">
 <tu align="CENTER" aunit="PERIODTO" aunitvalue="20211231" class="NORMAL" height="30" valign="MIDDLE" width="162">2021년 12월 31일</tu>
 <td aupdatecont="N" class="NORMAL" height="30" valign="MIDDLE" width="220"> 까지</td>
 </tr>
 </tbody>,
 <tbody>
 <tr acopy="Y" adelete="Y">
 <td height="30" width="200">금융위원회</td>
 <td align="RIGHT" height="30" width="400"></td>
 </tr>
 <tr acopy="Y" adelete="Y">
 <td height="30" width="200">한국거래소 귀중</td>
 <td align="RIGHT" height="30" width="400">2022년     3월     8일</td>
 </tr>
 <tr acopy="Y" adelete="Y">
 <td height="30" width="200"></td>
 <td height="30" w